In [1]:
import operator
from math import floor
from collections import namedtuple, defaultdict
from typing import Type
from itertools import chain
from more_itertools import windowed

In [2]:
with open('../data/2024/day22.txt', 'r') as f:
    data = f.read()

In [3]:
secrets = list(map(int, data.splitlines()))

In [4]:
def pseudo_randomize(s):
    s = operator.xor(s * 64, s) % 16777216
    s = operator.xor(floor(s / 32), s) % 16777216
    s = operator.xor(s * 2048, s) % 16777216
    return s

In [5]:
# Part 1
def secret_at_t(s, t):
    for _ in range(t): s = pseudo_randomize(s)
    return s

print("Part 1:", sum([secret_at_t(s, 2000) for s in secrets]))

Part 1: 19458130434


In [6]:
# Part 2
BuyerPrice: Type = namedtuple('BuyerPrice', ('price', 'delta'))
buyers = defaultdict(list)

# Calculate the first 2000 secrets for every buyer
for i,s in enumerate(secrets):
    for t in range(2000):
        last_ones = s % 10
        s = pseudo_randomize(s)
        ones = s % 10
        buyers[i].append(BuyerPrice(price=ones, delta=ones-last_ones))

sequences = [defaultdict(int) for b in buyers]

# Hash sequences first occurrences by buyer using a sliding window
for buyer, _ in enumerate(buyers):
    for w in windowed(buyers[buyer], 4):
        seq, price = tuple(b.delta for b in w), w[3].price
        if seq in sequences[buyer]: continue
        sequences[buyer][seq] = price

best_sum = 0

# Check every distinct sequence across all buyers and sum
for seq in set(chain(*sequences)):
    new_sum = sum([sequences[i].get(seq,0) for i in buyers])
    if new_sum > best_sum:
        best_sum = new_sum

print("Part 2:", best_sum)

Part 2: 2130
